In [1]:
import sys
from dotenv import load_dotenv
from crypto import convertDataToJSON, pinJSONtoIPFS, initContract, w3
from pprint import pprint

In [2]:
registerApt = initContract()

In [3]:
def createAptReport():
    time = input("Date of the auction: ")
    description = input("Description of the apartment: ")
    token_id = int(input("Token ID: "))

    json_data = convertDataToJSON(time, description)
    report_uri = pinJSONtoIPFS(json_data)

    return token_id, report_uri


def reportApt(token_id, report_uri):
    tx_hash = registerApt.functions.reportApt(token_id, report_uri).transact(
        {"from": w3.eth.accounts[0]}
    )
    receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    return receipt


def getAptReports(token_id):
    apt_filter = registerApt.events.Apt.createFilter(
        fromBlock="0x0", argument_filters={"token_id": token_id}
    )
    return apt_filter.get_all_entries()


In [ ]:
option = input("Would you like to get or report an apartment listing?")

if option == "report":
    token_id, report_uri = createAptReport()

    receipt = reportApt(token_id, report_uri)

    pprint(receipt)
    print("Report IPFS Hash:", report_uri)

if option == "get":
    token_id = int(sys.argv[2])

    apt = registerApt.functions.apts(token_id).call()
    reports = getAptReports(token_id)

    pprint(reports)
    print("APT", apt[0], "has been in", apt[1], "apartments.")